In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import re

In [3]:
# Cook soup
url = "https://www.metatopos.eu/belgcombiN.html"
r = requests.get(url)
print(url, r.status_code)
content = r.text
#print(content)
soup = BeautifulSoup(content, "html.parser")

https://www.metatopos.eu/belgcombiN.html 200


In [4]:
# Extract dat
post_data = dict()
for tr in soup.find_all("tr"):
    # Get data
    tr_data = []
    tds = tr.findChildren()
    for td in tds:
        tr_data.append(td.text)
    #print(tr_data)
    # Store data
    pcode = tr_data[0]
    if re.match("^[0-9]{4}$", pcode):
        if pcode not in post_data.keys():
            post_data[pcode] = dict()
            post_data[pcode]["city"] = tr_data[1]
            post_data[pcode]["city_alt"] = None
            if tr_data[2] != "":
                post_data[pcode]["municipality"] = [tr_data[2]]
            else:
                post_data[pcode]["municipality"] = None
        else:
            if tr_data[-1].find(post_data[pcode]["city"]) != -1:
                post_data[pcode]["city_alt"] = tr_data[1]
            elif post_data[pcode]["municipality"] == None:
                post_data[pcode]["municipality"] = [tr_data[2]]
            else:
                post_data[pcode]["municipality"].append(tr_data[2])

In [5]:
# Write to json
pcd_json_file = "..\data\\temp_postal_code_data.json"
with open(pcd_json_file, "w", encoding="utf-8") as pcd_json:
    pcd_json.write(json.dumps(post_data, indent=4))

In [6]:
pc_data_extra = pd.read_csv("..\data\zipcode_Belgium\zipcode-belgium.csv")
display(pc_data_extra.head())

,postalCode,municipality,longitude,latitude
0,1000,Bruxelles,4.351697,50.846557
1,1020,Laeken,4.348713,50.883392
2,1030,Schaerbeek,4.373712,50.867604
3,1040,Etterbeek,4.389510,50.836851
4,1050,Ixelles,4.381571,50.822285


In [7]:
lat = pc_data_extra.query('postalCode == 1050 & municipality == "Ixelles"')["latitude"].values[0]
lng = pc_data_extra.query('postalCode == 1050 & municipality == "Ixelles"')["longitude"].values[0]
print(lat, lng)

50.8222854 4.3815707


In [8]:
# Save as CSV from dictionary (open list)
pcdata_csv = "..\data\_postal_code_data.csv"
with open(pcdata_csv, "w", encoding="utf-8") as pcd_csv:
    header = ["postalCode", "city", "municipality", "latitude", "longitude", "cityAltName"]
    pcd_csv.write(",".join(header) + "\n")
    for code in post_data.keys():
        for m in post_data[code]["municipality"]:
            city = post_data[code]["city"]
            muni = m
            city_alt = str(post_data[code]["city_alt"])
            if code == "1050" and m == "Elsene":
                city = "Ixelles"
                muni = "Ixelles"
                city_alt = "Elsene"
            lat = pc_data_extra.query('postalCode == 1050 & municipality == "Ixelles"')["latitude"].values[0]
            lng = pc_data_extra.query('postalCode == 1050 & municipality == "Ixelles"')["longitude"].values[0]
            
            line = [code, city, muni, str(lat), str(lng), city_alt]
            pcd_csv.write(",".join(line) + "\n")